In [ ]:
import os
import glob
import pickle
###
import pandas as pd
import numpy as np
import MyFuncs as mf
import configparser as cnfp
###
from obspy import read_events
from datetime import datetime
from datetime import timedelta

In [ ]:
config_path = 'config.ini'
config = cnfp.ConfigParser(interpolation=cnfp.ExtendedInterpolation())
config.read(config_path)
#
pick_dir = config.get('Path', 'picks')
coord_path = config.get('Path', 'coordinate')
cat_path = config.get('Path', 'catalog')

residuals_threshold = config.get('Params', 'residuals_threshold_1')
residuals_threshold = int(residuals_threshold)

pick_name_pattern = config.get('File.Name.Pattern', 'picks')
residual_name_pattern = config.get('File.Name.Pattern', 'residual')

In [ ]:
path_to_residuals = config.get('Path', 'residuals')
os.makedirs(path_to_residuals, exist_ok=True)

In [ ]:
df = pd.read_csv(coord_path)
station2sensor = df.set_index('station')['sensor'].to_dict()
# print(station2sensor)

In [ ]:
catalog = read_events(cat_path)
catalog = catalog.filter('used_station_count > 0')
# print(catalog)

In [ ]:
# Checking Catalog
lst = []
for ev in catalog:
    for pick_ev in ev.picks:
        lst.append(pick_ev.phase_hint)
[_ for _ in lst if _=='']

In [ ]:
from seisbench.util.annotations import PickList, ClassifyOutput

def read_picks_of_date(path, fname):
    lst_file = glob.glob(f'{path}/{fname}')
    picks = PickList()
    l = len(lst_file)
    for indx, fname in enumerate(lst_file):
        # print(indx, 'of', l)
        with open(fname, 'rb') as fileObj:
            picks += pickle.load(fileObj)
    return picks

def read_picks_of_date_2(path, fname):
    lst_file = glob.glob(f'{path}/{fname}')
    picks = PickList()
    l = len(lst_file)
    for indx, fname in enumerate(lst_file):
        # print(indx, 'of', l)
        with open(fname, 'rb') as fileObj:
            classify = pickle.load(fileObj)
            # print(classify)
            picks += classify.picks
    return picks

In [ ]:
dates_of_cat = list({ev.preferred_origin().time.strftime('%Y%m%d') for ev in catalog})
dates_of_cat = sorted(dates_of_cat)

In [ ]:
for dl_model in ['PhaseNet', 'EQTransformer']:#
    for date_of_cat in dates_of_cat:
        date_of_cat_end = datetime.strptime(date_of_cat, '%Y%m%d')
        date_of_cat_end = date_of_cat_end + timedelta(days=1)
        date_of_cat_end =  date_of_cat_end.strftime('%Y%m%d')
        #
        fname = pick_name_pattern.format(
                net_sta='*',
                date_data=date_of_cat,
                model_name=dl_model,
                time_name='*'
                )
        picks_dl = read_picks_of_date_2(
            path=pick_dir, fname=fname
        )
        #
        cat_date = catalog.filter(
            f'time >= {date_of_cat}',
            f'time < {date_of_cat_end}'
        )
        for ev in cat_date:
            dic_result = {'autopick': [],
                          'cat_pick': [],
                          'cat_arrival': [],
                          'residual': []
                          }
            for pick_ev in ev.picks:
                if pick_ev.phase_hint.startswith('AM') or (pick_ev.phase_hint==''):
                    continue
                arriv_ev = mf.funcs.select_ArrivalOfPick(pick_ev, ev)
                sensor_id = station2sensor.get(pick_ev.waveform_id.station_code,
                                               None)
                if sensor_id is None:
                    print("There is no station, named", pick_ev.waveform_id.station_code)
                    continue
                # sensor_id = sensor_id.replace('t', '')
                picks_dl_station_phase = picks_dl.select(
                    trace_id=f'.{sensor_id}.',
                    phase=pick_ev.phase_hint[0].upper()
                )
                diff = [
                    p.peak_time-pick_ev.time
                    for p in picks_dl_station_phase
                ]
                diff = np.array(diff)
                mask = abs(diff) <= residuals_threshold
                residuals = diff[mask]
                picks_dl_selected = [
                    p
                    for (p, m) in zip(picks_dl_station_phase, mask)
                    if m
                ] 
                dic_result['cat_pick'].append(pick_ev)
                dic_result['residual'].append(residuals)
                dic_result['cat_arrival'].append(arriv_ev)
                dic_result['autopick'].append(picks_dl_selected)
            ###
            otime = ev.preferred_origin().time.strftime('%Y%m%dT%H%M%S')
            df_result = pd.DataFrame(dic_result)
            fname = residual_name_pattern.format(
                picker_name=dl_model,
                otime=otime
            )
            df_result.to_pickle(path=f'{path_to_residuals}/{fname}')
